In [6]:
import pandas as pd # Pandas helps you with managing Data (csv, excel, txt, etc.)
import numpy as np # Numpy helps you with Numerical Calculation (matrix, vectors operation, etc.)
import sklearn # This is a very popular package full of data mining algorithms.
from time import sleep

# This enables inline plotting in this notebook.
#%matplotlib inline
import matplotlib.pyplot as plt # A popular package for making visualization.

from bs4 import BeautifulSoup as bs # It makes web scraping easy.
import requests # It helps to retrieve contents of a URL
import re


url= "r'https://www.reddit.com/r/politicalfactchecking/"
request = requests.get(r'https://www.reddit.com/r/politicalfactchecking/')
r = request.text

#r = open('Part1')
soup = bs(r, "lxml")
siteTable = soup.find('div', attrs={'id': 'siteTable'})
threads = siteTable.findAll('div', attrs={'data-subreddit':'politicalfactchecking'})
urlofAllThreads = []


def verdictSimplifier(thS):
    if not (thS.find('span', attrs={'class':'linkflairlabel'})):
        tempVerdict = "None"
    else:
        tempVerdict = thS.find('span', attrs={'class':'linkflairlabel'}).contents[0]
    return tempVerdict

def threadDataExtractor(th, i):
    thread_id = th['id']
    time = th.time['datetime']
    author = th['data-author']
    vote_count = th.find('div', attrs={'class':'score unvoted'}).contents[0]
    comments = re.sub("\D", "", (th.find('a', attrs={'data-event-action':'comments'}).contents[0]))
    verdict = verdictSimplifier(th)
    title = th.find('a', attrs={'class':'title may-blank '}).contents[0]
    urlOfThread = 'https://www.reddit.com'+str((th.find('a', attrs = {'class': 'thumbnail self may-blank '})).get('href'))
    urlofAllThreads.append(urlOfThread)
    tData.loc[i+1] = [thread_id, time, author, vote_count, comments, verdict, title, urlOfThread]

tData = pd.DataFrame(columns=['ID','Time', 'Author', 'vote_count', 'Number_of_Comments', 'Verdict', 'Title', 'URL'])
for i,data in enumerate(threads):
    temp = threads[i]
    threadDataExtractor(temp, i)
tData.to_csv('ghoshal_torumoy_part1.csv')




In [ ]:
def cTextMerger(cThs):
    cText = cThs.find('div', attrs = {'class':'md'})
    cParagraphs = cText.findAll('p')
    blockQuote = cText.findAll('blockquote')
    cTable = cText.findAll('pre')
    clearText = ""
    for i,text in enumerate(cParagraphs):
        clearText = clearText + str(text.contents[0]) + '\n\n'
    for i, text in enumerate(blockQuote):
        clearText = clearText + str(text.contents[0]) + '\n\n'
    for i, text in enumerate(cTable):
        clearText = clearText + str(text.contents[0]) + '\n\n'
    return clearText

def parentID(cThs):
    if not cThs.find('div', attrs={'class':'child'}).contents:
        pid = "root"
    else:
        pid = str(cThs.find('p').a['name'])
    return(pid)

def cID(cThs):
    if not cThs['id']:
        cid = "None"
    else:
        cid = cThs['id']
    return(cid)

def commentInfoExtractor(cTh, i, num):
    thread_id = cID(cTh)
    author = cTh['data-author']
    points = re.sub("\D","",(cTh.find('span', attrs={'class':'score unvoted'}).contents[0]))
    time = cTh.time['datetime']
    parent_ID = parentID(cTh)
    text = cTextMerger(cTh)
    permalink = (cTh.find('a', attrs={'data-event-action':'permalink'}))['href']
    cData.loc[i+1+num] = [thread_id, author, points, time, parent_ID, text, permalink]

cData = pd.DataFrame(columns=['Comment_ID', 'Author', 'Points', 'Time', 'Parent_id', 'Text', 'Permalink'])

def cThreadGeneratorForComments(siteName):
    cRequest = requests.get(siteName)
    cR = cRequest.text
    #cR = open(siteName)
    cSoup = bs(cR, "lxml")
    cSiteTable = cSoup.find('div', attrs={'class':'sitetable nestedlisting'})
    cThreads = cSiteTable.findAll('div', attrs={'data-subreddit-fullname':'t5_2uvnw'})
    return cThreads

def cDataGenerator(cThreadTemp, num):
    for i,data in enumerate(cThreadTemp):
        temp = cThreadTemp[i]
        commentInfoExtractor(temp, i, num)


num = 0
for i,temp in enumerate(urlofAllThreads):
    sleep(5)
    cThreadData = cThreadGeneratorForComments(urlofAllThreads[i])
    cDataGenerator(cThreadData, num)
    num = num + len(cThreadData)

cData.to_csv('ghoshal_torumoy_part2.csv')
